In [1]:
import pandas as pd

In [2]:
# =========================
# 1) Load & clean
# =========================
df = pd.read_excel("./WindPowerForecastingData.xlsx")
df["DATETIME"] = pd.to_datetime(df["TIMESTAMP"], format="%Y%m%d %H:%M", errors="coerce")
df = df.sort_values("DATETIME").reset_index(drop=True)

TARGET_COL = "TARGETVAR"  # change this to actual target column name
df_clean = df.drop(columns=[c for c in ["TIMESTAMP","DATETIME"] if c in df.columns]).copy()
cols = [TARGET_COL] + [c for c in df_clean.columns if c != TARGET_COL]
df_clean = df_clean[cols].astype(float)

##############3
n_before = len(df_clean)
df_clean = df_clean.dropna(subset=[TARGET_COL]).copy()
print(f"Dropped {n_before - len(df_clean)} rows with missing target.")
##############

print (df_clean.head())
# =========================
# 2) Chronological split
# =========================
n = len(df_clean)
train_n = int(1 * n)
train_df = df_clean.iloc[:train_n].copy()
test_df  = df_clean.iloc[train_n:].copy()
print(".......................")
print(train_df[[TARGET_COL]])
y_train = train_df[[TARGET_COL]].values
print(".....y train ..................")
print(y_train)
X_train = train_df.drop(columns=[TARGET_COL]).values
print(".....x train ..................")
print(X_train)
y_test  = test_df[[TARGET_COL]].values
X_test  = test_df.drop(columns=[TARGET_COL]).values

Dropped 35 rows with missing target.
   TARGETVAR       U10       V10      U100      V100
0   0.000000  2.124600 -2.681966  2.864280 -3.666076
1   0.054879  2.521695 -1.796960  3.344859 -2.464761
2   0.110234  2.672210 -0.822516  3.508448 -1.214093
3   0.165116  2.457504 -0.143642  3.215233 -0.355546
4   0.156940  2.245898  0.389576  2.957678  0.332701
.......................
       TARGETVAR
0       0.000000
1       0.054879
2       0.110234
3       0.165116
4       0.156940
...          ...
16771   0.034301
16772   0.104407
16773   0.114839
16774   0.162579
16775   0.096232

[16765 rows x 1 columns]
.....y train ..................
[[0.        ]
 [0.05487912]
 [0.110234  ]
 ...
 [0.11483883]
 [0.1625787 ]
 [0.09623155]]
.....x train ..................
[[ 2.12460014 -2.68196637  2.86427959 -3.66607576]
 [ 2.52169465 -1.79696009  3.34485867 -2.46476146]
 [ 2.67220986 -0.82251622  3.50844802 -1.21409294]
 ...
 [-3.43336055  2.45759889 -4.79611597  3.40720263]
 [-3.22426125  2.04905457 -4

In [ ]:
import torch
import numpy as np
from sklearn.metrics import mean_squared_error
import math

# Load the full model
model = torch.load("glstm_noclass_best.pth")
print(model)
# model.eval()

# Convert data to tensor
X_tensor = torch.tensor(X_train, dtype=torch.float32)
y_tensor = torch.tensor(y_train, dtype=torch.float32)

# Run predictions
with torch.no_grad():
    y_pred = model(X_tensor).squeeze().numpy()

# Compute RMSE
rmse = math.sqrt(mean_squared_error(y, y_pred))
print(f"RMSE on entire dataset: {rmse:.4f}")


UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL torch.nn.modules.rnn.LSTM was not an allowed global by default. Please use `torch.serialization.add_safe_globals([torch.nn.modules.rnn.LSTM])` or the `torch.serialization.safe_globals([torch.nn.modules.rnn.LSTM])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about t